# Finding Volume of a Cell by sampling simulation.



In [ ]:
from IPython.display import IFrame
IFrame(src="https://www.youtube.com/embed/VLn59FSc4GA", width=560, height=340)

This first stage sets up the geometry and materials for the simulation.

In [ ]:
import openmc


# MATERIALS

mat_iron = openmc.Material()
mat_iron.set_density('g/cm3', 7.75)
mat_iron.add_element('Fe', 1.0, percent_type='wo')
mat_iron.id = 1 # the id is set so that is can be accessed later from the volume calculation
my_materials = openmc.Materials([mat_iron])


# GEOMETRY

# surfaces
outer_surface = openmc.Sphere(r=100, boundary_type='vacuum')

# cells
vessel_region = -outer_surface
vessel_cell = openmc.Cell(region=vessel_region)
vessel_cell.id = 1 # the id is set so that is can be accessed later from the volume calculation
vessel_cell.fill = mat_iron

universe = openmc.Universe(cells=[vessel_cell])
my_geometry = openmc.Geometry(universe)


# SIMULATION SETTINGS

# Instantiate a Settings object
my_settings = openmc.Settings()
batches = 10
my_settings.batches = batches
my_settings.inactive = 0
my_settings.particles = 10000
my_settings.run_mode = 'fixed source'

# Create a DT point source
my_source = openmc.Source()
my_source.space = openmc.stats.Point((0, 0, 0))
my_source.angle = openmc.stats.Isotropic()
my_source.energy = openmc.stats.Discrete([14e6], [1])
my_settings.source = my_source

This sets up the damage energy tally using the MT number 444. A list of MT numbers including their reaction discription can be found [here](https://t2.lanl.gov/nis/endf/mts.html).

This runs the simulation.

In [ ]:
model = openmc.model.Model(my_geometry, my_materials, my_settings)
!rm *.h5
sp_filename = model.run()

This section carries out a volume calculation for the CSG cell. Particles are created randomly within the bounding box (of known volume). Upon creation of particles a check is carried out to see if the particle is also in the CSG volume. The ratio of the particles made in the bounding box to the particles made in the CSG cell is then used to determine the volume of the CSG cell. And error for the CSG cell is also determined.

In [ ]:
# volume calculates for materials require a bounding box
# you can make the simulation more efficient if the bounding box volume is set to twice the volume of the cell volume
lower_left, upper_right = my_geometry.bounding_box()

material_vol_calc = openmc.VolumeCalculation([mat_iron], 100000, lower_left, upper_right)

cell_vol_calc = openmc.VolumeCalculation([vessel_cell], 100000)

settings = openmc.Settings()
settings.volume_calculations = [cell_vol_calc, material_vol_calc]
settings.run_mode = 'volume'
settings.export_to_xml()

!rm *.h5
openmc.run()


This next section extracts the results from the output files

In [ ]:

cell_vol_calc_results = openmc.VolumeCalculation.from_hdf5('volume_1.h5')
print('\n cell volume', cell_vol_calc_results.volumes[1], 'cm3')  # the [1] is the ID of the cell

material_vol_calc_results = openmc.VolumeCalculation.from_hdf5('volume_2.h5')
print(' material volume', material_vol_calc_results.volumes[1], 'cm3')  # the [1] is the ID of the material

# the cell_vol_calc_results are combined with errors, you can access the
# result on its own using the .nominal_value method

volume_of_cell_1 = cell_vol_calc_results.volumes[1].nominal_value


As we know th volume of the cell in this case (4/3PiR^3) then we can check the accuracty of the answer against this known value.

This method is intended to be used when the cell is more complex and the volume is not known. However for this example we wanted to be able to check.

Now try reducing the number of particles and see how the answer varies in accuracy.

Therefore, as the total number of atoms and the total number of displacements is known, DPA can be found.

**Learning Outcomes for Task**

- The volume of a cell or a material can be found using the stochastic volume method.
- Volume found in this way is accompanied by a uncertainty.